In [30]:
from pathlib import Path
from openai import OpenAI
import os
import json
import pandas as pd

file_path = os.path.join('for_create_sample_data', '【财通证券】酒店餐饮年报及一季报总结：酒店业绩分化，餐饮相对韧性.docx')

In [31]:
client = OpenAI(
    api_key = "sk-ekZUpvVQ8W7cHtcKWO947MpC4C0v9a5B5XUgyKRijAUeHeSo",
    base_url = "https://api.moonshot.cn/v1",
)

# xlnet.pdf 是一个示例文件, 我们支持 pdf, doc 以及图片等格式, 对于图片和 pdf 文件，提供 ocr 相关能力
file_object = client.files.create(file=Path(file_path), purpose="file-extract")

# 获取结果
# file_content = client.files.retrieve_content(file_id=file_object.id)
# 注意，之前 retrieve_content api 在最新版本标记了 warning, 可以用下面这行代替
# 如果是旧版本，可以用 retrieve_content
file_content = client.files.content(file_id=file_object.id).text

content = """
请从文件中抽取20条QA对，输出格式为json。如下：
[
    {
        "question": "皇玺集团在2023年第一季度的未经审核收益是多少？",
        "answer": "约6.8百万港元"
    },
    {
        "question": "皇玺集团在2023年第一季度录得的公司拥有人应占未经审核亏损及全面亏损总额是多少？",
        "answer": "约4.7百万港元"
    },
    {
        "question": "皇玺集团的注册办事处地址在哪里？",
        "answer": "Second Floor, Century Yard, Cricket Square, P.O. Box 902, Grand Cayman, KY1–1103, Cayman Islands"
    },
    {
        "question": "皇玺集团在2023年第一季度的财务成本是多少？",
        "answer": "约0.3百万港元"
    },
    {
        "question": "皇玺集团在2023年第一季度的员工成本是多少？",
        "answer": "约4.2百万港元"
    }
]
"""

# 把它放进请求中
messages = [
    {
        "role": "system",
        "content": "你是QA输出小助手，根据文档内容，从中抽取QA对。注意，你只需要输出json内容，其它的无需输出",
    },
    {
        "role": "system",
        "content": file_content,
    },
    {"role": "user", "content": content},
]

# 然后调用 chat-completion, 获取 Kimi 的回答
completion = client.chat.completions.create(
    model="moonshot-v1-32k",
    messages=messages,
    temperature=0.3,
)

result  = completion.choices[0].message

KeyboardInterrupt: 

In [28]:
result_data = json.loads(result.content)

In [29]:
# 创建 DataFrame
df_result = pd.DataFrame(result_data)

In [26]:
# # 定义文件路径
# file_path = 'output.txt'
# 
# # 将字符串写入文件
# with open(file_path, 'w', encoding='utf-8') as file:
#     file.write(data)
# 
# print(f"字符串已保存到 {file_path}")

字符串已保存到 output.txt
